In [1]:
from rdkit.Chem import AllChem
from scipy import stats
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from scipy import stats
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem.Scaffolds import MurckoScaffold
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef,confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns
import time
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

In [2]:
import pandas as pd
import numpy as np

In [3]:
df_data = pd.read_csv("./assay_data_filtered_88_inchi.csv") 
assay_list= df_data.columns.to_list()[1:-1]
df_data = df_data.drop(columns= ["smiles_r"])
df_data

,InChICode_standardised,588458,588334,2642,2156,2330,2216,743015,504444,894,...,652104,720579,720533,720542,720580,720504,720532,1159524,1117304,1117305
0,InChI=1S/C14H13N5O5S2/c1-2-5-3-25-12-8(11(21)1...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,InChI=1S/C22H21NO2S/c23-20(21(24)25)16-26-22(1...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,InChI=1S/C15H8O7/c16-6-3-8-12(10(18)4-6)14(20)...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
3,InChI=1S/C8H10O4/c1-5(2)8(10)6(11-3)4-7(9)12-8...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,InChI=1S/C14H14O4S2/c15-5-7-19-13-11(17)9-3-1-...,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9871,InChI=1S/C11H8O3/c1-7(12)9-6-8-4-2-3-5-10(8)14...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
9872,InChI=1S/C15H18N2/c1-2-7-14-12(5-1)13-6-3-4-11...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
9873,InChI=1S/C33H44N4O4/c1-22-18-37(23(2)20-38)32(...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0
9874,InChI=1S/C18H16NO/c1-14-11-12-19(17-10-6-5-9-1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [4]:
len(assay_list)

88

In [5]:
assay_list[-1]

'1117305'

In [6]:
CP_features = pd.read_csv("./CP_filtered_9876.csv") 
CP_features_list= CP_features.columns.to_list()[:-1]
#CP_features = CP_features.to_numpy()
CP_features

,Cells_AreaShape_MedianRadius,Cells_Correlation_Correlation_DNA_AGP,Cells_Correlation_Correlation_ER_AGP,Cells_Correlation_K_AGP_DNA,Cells_Correlation_K_DNA_AGP,Cells_Correlation_K_DNA_ER,Cells_Correlation_K_DNA_Mito,Cells_Correlation_K_ER_DNA,Cells_Correlation_K_Mito_DNA,Cells_Correlation_K_RNA_DNA,...,Nuclei_Texture_SumVariance_DNA_10_0,Nuclei_Texture_SumVariance_ER_10_0,Nuclei_Texture_SumVariance_Mito_10_0,Nuclei_Texture_SumVariance_RNA_10_0,Nuclei_Texture_SumVariance_RNA_5_0,Nuclei_Texture_Variance_AGP_10_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_RNA_10_0,InChICode_standardised
0,0.963798,0.159010,0.047354,-0.137963,0.216676,0.144280,0.113497,-0.117675,-0.079211,-0.206469,...,-0.245524,-0.268078,-0.123740,0.122226,0.402861,-0.206448,-0.158874,-0.121066,0.185686,InChI=1S/C14H13N5O5S2/c1-2-5-3-25-12-8(11(21)1...
1,1.210207,0.367478,0.263613,0.083119,-0.181781,-0.118582,-0.041744,0.069925,0.043344,0.016976,...,-0.125642,0.168251,-0.116379,0.316108,0.299225,-0.173420,0.057710,0.010376,0.272074,InChI=1S/C22H21NO2S/c23-20(21(24)25)16-26-22(1...
2,0.581696,0.125987,0.092521,-0.021064,0.054373,-0.037538,0.053001,0.018777,-0.053614,0.075178,...,-0.059526,0.071001,-0.029977,-1.310799,-1.847892,-0.000630,0.055282,0.028152,-0.610017,InChI=1S/C15H8O7/c16-6-3-8-12(10(18)4-6)14(20)...
3,0.248304,-0.014260,-0.018056,-0.037718,0.063135,0.032092,-0.014411,-0.009945,-0.000591,-0.023499,...,-0.554508,-0.357952,-0.232800,-0.197858,-0.499390,-0.065841,-0.123535,-0.071141,-0.109201,InChI=1S/C8H10O4/c1-5(2)8(10)6(11-3)4-7(9)12-8...
4,-2.382596,0.342042,0.183357,-0.814742,1.361776,3.826898,3.387335,-0.507087,-0.559135,-0.432035,...,-2.017392,1.497684,0.264454,-0.562605,1.086744,0.311285,0.727071,0.874987,0.189458,InChI=1S/C14H14O4S2/c15-5-7-19-13-11(17)9-3-1-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9871,-0.000341,-0.022052,-0.049354,-0.077226,0.036142,0.035560,0.034507,-0.021980,-0.036769,-0.026607,...,-0.154942,-0.208285,0.094947,-0.064585,-0.062841,-0.050082,-0.050364,0.009238,-0.025615,InChI=1S/C11H8O3/c1-7(12)9-6-8-4-2-3-5-10(8)14...
9872,0.148294,0.043186,0.001665,-0.007765,-0.000027,-0.011346,0.075195,0.006984,-0.059737,-0.038276,...,-0.421050,-0.162980,-0.191575,-0.036555,-0.244200,-0.052731,-0.053152,-0.102096,0.074795,InChI=1S/C15H18N2/c1-2-7-14-12(5-1)13-6-3-4-11...
9873,0.043964,0.063290,0.016303,-0.056327,0.025226,0.010598,0.121048,-0.004245,-0.069040,-0.121069,...,0.805582,0.352569,-0.237808,0.390295,0.707784,0.231027,0.102048,-0.001059,0.298214,InChI=1S/C33H44N4O4/c1-22-18-37(23(2)20-38)32(...
9874,-0.080926,-0.094140,-0.105359,0.045901,-0.024562,0.026936,-0.022409,-0.024922,0.042600,-0.009710,...,-0.200248,-0.237660,-0.151925,-0.112267,-0.158519,-0.020584,-0.080274,-0.091421,-0.034917,InChI=1S/C18H16NO/c1-14-11-12-19(17-10-6-5-9-1...


In [7]:
mfp_features = pd.read_csv("./Mfp_filtered_9876.csv") 
mfp_features_list = mfp_features.columns.to_list()[:-1]
mfp_features['fp'] = [(x) for x in mfp_features.iloc[:, :-1].to_numpy()]
mfp_features = mfp_features[["InChICode_standardised", "fp"]]
mfp_features

,InChICode_standardised,fp
0,InChI=1S/C14H13N5O5S2/c1-2-5-3-25-12-8(11(21)1...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,InChI=1S/C22H21NO2S/c23-20(21(24)25)16-26-22(1...,"[1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, ..."
2,InChI=1S/C15H8O7/c16-6-3-8-12(10(18)4-6)14(20)...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,InChI=1S/C8H10O4/c1-5(2)8(10)6(11-3)4-7(9)12-8...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,InChI=1S/C14H14O4S2/c15-5-7-19-13-11(17)9-3-1-...,"[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
...,...,...
9871,InChI=1S/C11H8O3/c1-7(12)9-6-8-4-2-3-5-10(8)14...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9872,InChI=1S/C15H18N2/c1-2-7-14-12(5-1)13-6-3-4-11...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9873,InChI=1S/C33H44N4O4/c1-22-18-37(23(2)20-38)32(...,"[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9874,InChI=1S/C18H16NO/c1-14-11-12-19(17-10-6-5-9-1...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [8]:
assay_data= pd.merge(df_data, CP_features)
assay_data= pd.merge(assay_data, mfp_features)
assay_data

,InChICode_standardised,588458,588334,2642,2156,2330,2216,743015,504444,894,...,Nuclei_Texture_SumVariance_DNA_10_0,Nuclei_Texture_SumVariance_ER_10_0,Nuclei_Texture_SumVariance_Mito_10_0,Nuclei_Texture_SumVariance_RNA_10_0,Nuclei_Texture_SumVariance_RNA_5_0,Nuclei_Texture_Variance_AGP_10_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_Mito_10_0,Nuclei_Texture_Variance_RNA_10_0,fp
0,InChI=1S/C14H13N5O5S2/c1-2-5-3-25-12-8(11(21)1...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.245524,-0.268078,-0.123740,0.122226,0.402861,-0.206448,-0.158874,-0.121066,0.185686,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,InChI=1S/C22H21NO2S/c23-20(21(24)25)16-26-22(1...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.125642,0.168251,-0.116379,0.316108,0.299225,-0.173420,0.057710,0.010376,0.272074,"[1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, ..."
2,InChI=1S/C15H8O7/c16-6-3-8-12(10(18)4-6)14(20)...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,-0.059526,0.071001,-0.029977,-1.310799,-1.847892,-0.000630,0.055282,0.028152,-0.610017,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,InChI=1S/C8H10O4/c1-5(2)8(10)6(11-3)4-7(9)12-8...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,-0.554508,-0.357952,-0.232800,-0.197858,-0.499390,-0.065841,-0.123535,-0.071141,-0.109201,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,InChI=1S/C14H14O4S2/c15-5-7-19-13-11(17)9-3-1-...,1.0,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,...,-2.017392,1.497684,0.264454,-0.562605,1.086744,0.311285,0.727071,0.874987,0.189458,"[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9871,InChI=1S/C11H8O3/c1-7(12)9-6-8-4-2-3-5-10(8)14...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.154942,-0.208285,0.094947,-0.064585,-0.062841,-0.050082,-0.050364,0.009238,-0.025615,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9872,InChI=1S/C15H18N2/c1-2-7-14-12(5-1)13-6-3-4-11...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.421050,-0.162980,-0.191575,-0.036555,-0.244200,-0.052731,-0.053152,-0.102096,0.074795,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9873,InChI=1S/C33H44N4O4/c1-22-18-37(23(2)20-38)32(...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.805582,0.352569,-0.237808,0.390295,0.707784,0.231027,0.102048,-0.001059,0.298214,"[1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9874,InChI=1S/C18H16NO/c1-14-11-12-19(17-10-6-5-9-1...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.200248,-0.237660,-0.151925,-0.112267,-0.158519,-0.020584,-0.080274,-0.091421,-0.034917,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
def TanimotoSimilarity(ex1, ex2, attrs):
  """
    >>> v1 = [0,1,0,1]
    >>> v2 = [1,0,1,0]
    >>> TanimotoDist(v1,v2,range(4))
    1.0
    """
  inter = 0.0
  unin = 0.0
  for i in attrs:
    if (ex1[i] or ex2[i]):
      unin += 1
      if (ex1[i] and ex2[i]):
        inter += 1
  if (unin != 0.0):
    return (inter / unin)
  else:
    return 0

In [10]:
def BulkTanimoto(siv1, sivs):
    res = []
    for i in range(len(sivs)): 
        siv2 = sivs[i]
        simVal = TanimotoSimilarity(siv1, siv2, range(len(siv1)))
        res.append(simVal)
    return res

In [11]:
[1,2, 3,4][::-1]

[4, 3, 2, 1]

In [12]:
#Find similarity between each test compounds and train set compounds
#mean of the 5 highest tanimoto similarity of MFP keys, all, actives, inactives
#perc of compound with MFP TS>0.55, all, actives, inactives
neighbours_limit = 5
def findsimscore(assay, train, test): 
    
    ts_nearest_list =[]
    ts_nearest_actives_list=[]
    ts_nearest_inactives_list=[]
    perc_list=[]
    perc_actives_list=[]
    perc_inactives_list=[]
    
    for index, row in test.iterrows():

        fps_test = row["fp"]

        temp_train = pd.DataFrame()
        temp_train= train

        fps_temp_train=[(x) for x in temp_train["fp"]]
        ts_nearest = []
        ts_nearest = BulkTanimoto(fps_test ,fps_temp_train)

        #Add all values with Activity
        df_temp = pd.DataFrame([temp_train[[assay]].values.flatten(), ts_nearest]).T
        df_temp.columns = ["Activity", "TS_similarity"]
        
        #Calculate overall percetages
        count = len([i for i in df_temp["TS_similarity"].values if i > 0.55])
        total= len(df_temp)
        perc= float(count/total)  
        perc_list.append(perc)
       
        ts_nearest_ovarall = df_temp.sort_values(by="TS_similarity", ascending=False)["TS_similarity"][:neighbours_limit].mean()
        #print(ts_nearest_ovarall)
        ts_nearest_list.append(ts_nearest_ovarall)
        
        #Calculate percetages active compound
        count = len([i for i in df_temp[df_temp["Activity"]==1]["TS_similarity"].values if i > 0.55])
        total= (len(df_temp[df_temp["Activity"]==1]))
        perc_actives= float(count/total)
        perc_actives_list.append(perc_actives)
        
        ts_nearest_actives = df_temp[df_temp["Activity"]==1].sort_values(by="TS_similarity", ascending=False)["TS_similarity"][:neighbours_limit].mean()
        #print(ts_nearest_actives)
        ts_nearest_actives_list.append(ts_nearest_actives)
        
        #Calculate percetages inactive compound
        count = len([i for i in df_temp[df_temp["Activity"]==0]["TS_similarity"].values if i > 0.55])
        total= (len(df_temp[df_temp["Activity"]==0]))
        perc_inactives= float(count/total)
        perc_inactives_list.append(perc_inactives)
        
        ts_nearest_inactives = df_temp[df_temp["Activity"]==0].sort_values(by="TS_similarity", ascending=False)["TS_similarity"][:neighbours_limit].mean()
        #print(ts_nearest_inactives)
        ts_nearest_inactives_list.append(ts_nearest_inactives)

    ts_nearest_list=np.array(ts_nearest_list)
    ts_nearest_list= ts_nearest_list.reshape(len(test))
    
    ts_nearest_actives_list=np.array(ts_nearest_actives_list)
    ts_nearest_actives_list= ts_nearest_actives_list.reshape(len(test))
    
    ts_nearest_inactives_list=np.array(ts_nearest_inactives_list)
    ts_nearest_inactives_list= ts_nearest_inactives_list.reshape(len(test))

    perc_list=np.array(perc_list)
    perc_list= perc_list.reshape(len(test))
    
    perc_actives_list=np.array(perc_actives_list)
    perc_actives_list= perc_actives_list.reshape(len(test))
    
    perc_inactives_list=np.array(perc_inactives_list)
    perc_inactives_list= perc_inactives_list.reshape(len(test))

    df_ts_nearest = pd.DataFrame({"ts_nearest": ts_nearest_list, "perc": perc_list,
                              "ts_nearest_actives": ts_nearest_actives_list, "perc_actives": perc_actives_list,
                              "ts_nearest_inactives": ts_nearest_inactives_list, "perc_inactives": perc_inactives_list})
    return df_ts_nearest

In [13]:
#Find similarity between each test compounds and train set compounds
#mean of the 5 highest PC keys, all, actives, inactives
#perc of compound with PC TS>0.55, all, actives, inactives

neighbours_limit = 5
def findpearsonscore(assay, train, test): 
    pc_highest_list =[]
    pc_highest_actives_list=[]
    pc_highest_inactives_list=[]
    perc_list=[]
    perc_actives_list=[]
    perc_inactives_list=[]
    
    for index, row in test.iterrows():

        fps_test = row[CP_features_list].to_numpy()

        temp_train = pd.DataFrame()
        temp_train= train

        fps_temp_train=[(x) for x in temp_train[CP_features_list].to_numpy()]
        pearson_s_list = []
        
        for n in range(len(fps_temp_train)):
            
            pearson= stats.pearsonr(fps_test, fps_temp_train[n])
            pearson_s_list.append(pearson[0])
            #mean of 15 positive correlations
            
        #Add all values with Activity
        df_temp = pd.DataFrame([temp_train[[assay]].values.flatten(), pearson_s_list]).T
        df_temp.columns = ["Activity", "PC_corr"]
        
        #Calculate overall percetages
        count = len([i for i in df_temp["PC_corr"].values if i > 0.55])
        total= len(df_temp)
        perc= float(count/total)  
        perc_list.append(perc)
       
        pc_highest_ovarall = df_temp.sort_values(by="PC_corr", ascending=False)["PC_corr"][:neighbours_limit].mean()
        #print(pc_highest_ovarall)
        pc_highest_list.append(pc_highest_ovarall)
        
        #Calculate percetages active compound
        count = len([i for i in df_temp[df_temp["Activity"]==1]["PC_corr"].values if i > 0.50])
        total= (len(df_temp[df_temp["Activity"]==1]))
        perc_actives= float(count/total)
        perc_actives_list.append(perc_actives)
        
        pc_highest_actives = df_temp[df_temp["Activity"]==1].sort_values(by="PC_corr", ascending=False)["PC_corr"][:neighbours_limit].mean()
        #print(pc_highest_actives)
        pc_highest_actives_list.append(pc_highest_actives)
        
        #Calculate percetages inactive compound
        count = len([i for i in df_temp[df_temp["Activity"]==0]["PC_corr"].values if i > 0.50])
        total= (len(df_temp[df_temp["Activity"]==0]))
        perc_inactives= float(count/total)
        perc_inactives_list.append(perc_inactives)
        
        pc_highest_inactives = df_temp[df_temp["Activity"]==0].sort_values(by="PC_corr", ascending=False)["PC_corr"][:neighbours_limit].mean()
        #print(ts_nearest_inactives)
        pc_highest_inactives_list.append(pc_highest_inactives)

    pc_highest_list=np.array(pc_highest_list)
    pc_highest_list= pc_highest_list.reshape(len(test))
    
    pc_highest_actives_list=np.array(pc_highest_actives_list)
    pc_highest_actives_list= pc_highest_actives_list.reshape(len(test))
    
    pc_highest_inactives_list=np.array(pc_highest_inactives_list)
    pc_highest_inactives_list= pc_highest_inactives_list.reshape(len(test))

    perc_list=np.array(perc_list)
    perc_list= perc_list.reshape(len(test))
    
    perc_actives_list=np.array(perc_actives_list)
    perc_actives_list= perc_actives_list.reshape(len(test))
    
    perc_inactives_list=np.array(perc_inactives_list)
    perc_inactives_list= perc_inactives_list.reshape(len(test))

    df_pc_highest = pd.DataFrame({"pc_highest": pc_highest_list, "perc": perc_list,
                              "pc_highest_actives": pc_highest_actives_list, "perc_actives": perc_actives_list,
                              "pc_highest_inactives": pc_highest_inactives_list, "perc_inactives": perc_inactives_list})
    return df_pc_highest

In [14]:
def plot_hist_ts(df):
    
    #f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw= {"height_ratios": (0.2, 1)})
    
    mean=df['ts_nearest'].mean()
    median=df['ts_nearest'].median()
    mode=df['ts_nearest'].mode().tolist()[0]
    
    
    #sns.boxplot(x=df["td_nearest"], ax=ax_box)
    #ax_box.axvline(mean, color='r', linestyle='--',  label='Mean')
    #ax_box.axvline(median, color='g', linestyle='-',  label='Median')
    #ax_box.axvline(mode, color='b', linestyle='-',  label='Mode')

    #sns.histplot(x=df["td_nearest"],  kde=True, ax=ax_hist)
    #ax_hist.axvline(mean, color='r', linestyle='--', label='Mean')
    #ax_hist.axvline(median, color='g', linestyle='-', label='Median')
    #ax_hist.axvline(mode, color='b', linestyle='-', label='Mode')

    #plt.legend()
    #ax_box.set(xlabel='')
    #plt.show()
    
    return mean, median, mode

def plot_hist_pc(df):
    
    #f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, gridspec_kw= {"height_ratios": (0.2, 1)})
    
    mean=df['pc_highest'].mean()
    median=df['pc_highest'].median()
    mode=df['pc_highest'].mode().tolist()[0]
    
    
    #sns.boxplot(x=df["pc_highest"], ax=ax_box)
    #ax_box.axvline(mean, color='r', linestyle='--',  label='Mean')
    #ax_box.axvline(median, color='g', linestyle='-',  label='Median')
    #ax_box.axvline(mode, color='b', linestyle='-',  label='Mode')

    #sns.histplot(x=df["pc_highest"],  kde=True, ax=ax_hist)
    #ax_hist.axvline(mean, color='r', linestyle='--', label='Mean')
    #ax_hist.axvline(median, color='g', linestyle='-', label='Median')
    #ax_hist.axvline(mode, color='b', linestyle='-', label='Mode')

    #plt.legend()
    #ax_box.set(xlabel='')
    #plt.show()
    
    return mean, median, mode

In [15]:
#Nested Cross validation
#stat_list_batch: contains assay number, mean, median, mode of tanimoto distance in that cycle, mcc and auc of that cycle
#detail_list_batch contains assay number, each inchi and its proba, pred, true values and ts
#pred_list_batch contains proba, pred and true values

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.under_sampling import RandomUnderSampler 

from collections import Counter
from numpy import argmax

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from scipy.stats import ks_2samp 
from scipy.stats import randint
from sklearn.model_selection import cross_val_predict

def my_function_nested_rhs(assay, cycle):   
        
    detail_list_batch_train = []

    X_fp = df_assay_train.loc[:, "fp"].to_numpy()
    X_CP = df_assay_train[CP_features_list].to_numpy()
    
    #print(X)
    y = df_assay_train[assay].to_numpy()
    #print(y)
    
    
    #Split
    outercv = StratifiedKFold(n_splits=5,shuffle=True, random_state=cycle)
   
    for i, (train_index, test_index) in enumerate(outercv.split(X_fp, y)):
        
        train= df_assay_train.iloc[train_index]
        test= df_assay_train.iloc[test_index]
        #print(train)
        #print(test)

        #df_ts_perc = findsimsperc(assay, train, test) 
        #print
        
        df_ts_highest = findsimscore(assay, train, test)
        df_pc_highest = findpearsonscore(assay, train, test)    
        
        mean_ts, median_score, mode_score= plot_hist_ts(df_ts_highest)
        mean_pc, median_score, mode_score= plot_hist_pc(df_pc_highest)

        #combine results
        for InChICode_standardised, ts, ts_perc, ts_actives, ts_perc_actives, ts_inactives, ts_perc_inactives, pc, pc_perc, pc_actives, pc_perc_actives, pc_inactives, pc_perc_inactives  in zip(test.InChICode_standardised, 
                                                df_ts_highest.ts_nearest, df_ts_highest.perc, df_ts_highest.ts_nearest_actives, df_ts_highest.perc_actives,  df_ts_highest.ts_nearest_inactives, df_ts_highest.perc_inactives,   
                                                  df_pc_highest.pc_highest, df_pc_highest.perc, df_pc_highest.pc_highest_actives, df_pc_highest.perc_actives, df_pc_highest.pc_highest_inactives, df_pc_highest.perc_inactives):
            detail_list_batch_train.append([assay, InChICode_standardised, ts, ts_perc, ts_actives, ts_perc_actives, 
                                            ts_inactives, ts_perc_inactives, pc, pc_perc, pc_actives, 
                                            pc_perc_actives, pc_inactives, pc_perc_inactives, "Training"])
    
    return detail_list_batch_train

In [16]:
#Predict External Test Set
#detail_list_batch contains assay number, each inchi and its proba, pred, true values and ts

def my_function_heldout_rhs(assay, cycle):   
        
        detail_list_batch_test = []

        train= df_assay_train
        test= df_assay_test

        df_ts_highest = findsimscore(assay, train, test)
        df_pc_highest = findpearsonscore(assay, train, test)    
        
        mean_ts, median_score, mode_score= plot_hist_ts(df_ts_highest)
        mean_pc, median_score, mode_score= plot_hist_pc(df_pc_highest)
        
        #combine results
        for InChICode_standardised, ts, ts_perc, ts_actives, ts_perc_actives, ts_inactives, ts_perc_inactives, pc, pc_perc, pc_actives, pc_perc_actives, pc_inactives, pc_perc_inactives  in zip(test.InChICode_standardised, 
                                                df_ts_highest.ts_nearest, df_ts_highest.perc, df_ts_highest.ts_nearest_actives, df_ts_highest.perc_actives,  df_ts_highest.ts_nearest_inactives, df_ts_highest.perc_inactives,   
                                                  df_pc_highest.pc_highest, df_pc_highest.perc, df_pc_highest.pc_highest_actives, df_pc_highest.perc_actives, df_pc_highest.pc_highest_inactives, df_pc_highest.perc_inactives):
            detail_list_batch_test.append([assay, InChICode_standardised, ts, ts_perc, ts_actives, ts_perc_actives, 
                                           ts_inactives, ts_perc_inactives, pc, pc_perc, pc_actives, pc_perc_actives, 
                                           pc_inactives, pc_perc_inactives, "HeldOut"])
    
        return detail_list_batch_test

In [17]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split

master_detail_list=[]
for assay in tqdm(assay_list[:]):
    
    print("Assay ", assay )
    df_assay= assay_data[pd.notnull(assay_data[assay])]

    print("Total Compounds ", len(df_assay) )

    df_assay_inactive = df_assay[df_assay[assay] == 0]
    inactives_n=len(df_assay_inactive)
    print("Inactive Compounds ", inactives_n)

    df_assay_active = df_assay[df_assay[assay] == 1]
    actives_n=len(df_assay_active)
    print("Active Compounds ", actives_n)

    df_assay_train, df_assay_test= train_test_split(df_assay, stratify=df_assay[[assay]], test_size=0.20, random_state=42)
    
    print("Perform nested cross validation on training set")
    detail_list_train =  my_function_nested_rhs(assay, 42)
   
    
    print("Predictions on held out set")
    detail_list_test = my_function_heldout_rhs(assay, 42)

    
    master_detail_list.append(detail_list_train) 
    master_detail_list.append(detail_list_test) 
     

  0%|                                                    | 0/88 [00:00<?, ?it/s]

Assay  588458
Total Compounds  448
Inactive Compounds  336
Active Compounds  112
Perform nested cross validation on training set
Predictions on held out set


  1%|▌                                           | 1/88 [00:17<24:43, 17.05s/it]

Assay  588334
Total Compounds  528
Inactive Compounds  396
Active Compounds  132
Perform nested cross validation on training set
Predictions on held out set


  2%|█                                           | 2/88 [00:39<29:22, 20.49s/it]

...

 98%|████████████████████████████████████████ | 86/88 [1:10:29<00:42, 21.25s/it]

Assay  1117304
Total Compounds  386
Inactive Compounds  260
Active Compounds  126
Perform nested cross validation on training set
Predictions on held out set


 99%|████████████████████████████████████████▌| 87/88 [1:10:42<00:18, 18.71s/it]

Assay  1117305
Total Compounds  359
Inactive Compounds  247
Active Compounds  112
Perform nested cross validation on training set
Predictions on held out set


100%|█████████████████████████████████████████| 88/88 [1:10:53<00:00, 48.33s/it]


In [18]:
flatList_master_detail_list = [ item for elem in master_detail_list for item in elem]
list_of_lists_df = pd.DataFrame(flatList_master_detail_list,columns=["assay","InChICode_standardised",
                                                                    "ts", "ts_perc", "ts_actives", "ts_perc_actives", 
                                                                     "ts_inactives", "ts_perc_inactives", "pc", "pc_perc", "pc_actives", "pc_perc_actives", 
                                                                     "pc_inactives", "pc_perc_inactives", 
                                                                     "Data"])
list_of_lists_df

,assay,InChICode_standardised,ts,ts_perc,ts_actives,ts_perc_actives,ts_inactives,ts_perc_inactives,pc,pc_perc,pc_actives,pc_perc_actives,pc_inactives,pc_perc_inactives,Data
0,588458,"InChI=1S/C19H19N3O5S3/c23-16-6-5-13(30(25,26)2...",0.446814,0.000000,0.396191,0.000000,0.433559,0.000000,0.442869,0.000000,0.416542,0.000000,0.430447,0.000000,Training
1,588458,InChI=1S/C9H8ClN3S/c1-6-11-12-9(14)13(6)8-4-2-...,0.443323,0.000000,0.378563,0.000000,0.414811,0.000000,0.689046,0.139860,0.662077,0.222222,0.678060,0.214953,Training
2,588458,InChI=1S/C20H20N2O4/c1-2-25-20(24)17-12-18-16(...,0.440373,0.000000,0.373476,0.000000,0.428608,0.000000,0.729628,0.143357,0.676574,0.152778,0.719979,0.186916,Training
3,588458,InChI=1S/C26H24N2O5S/c1-33-19-11-9-18(10-12-19...,0.478412,0.006993,0.332100,0.000000,0.478412,0.009346,0.536519,0.000000,0.458671,0.013889,0.526597,0.018692,Training
4,588458,InChI=1S/C19H21N3O3S/c1-13-10-16-11-15(4-9-19(...,0.473341,0.000000,0.401539,0.000000,0.420060,0.000000,0.707402,0.055944,0.599794,0.055556,0.673625,0.074766,Training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50191,1117305,"InChI=1S/C7H8ClN3O4S2/c8-4-1-5-7(2-6(4)16(9,12...",0.498732,0.006969,0.479915,0.022222,0.314529,0.000000,0.645185,0.045296,0.605005,0.111111,0.628868,0.055838,HeldOut
50192,1117305,InChI=1S/C14H9I3O4/c15-9-6-8(1-2-12(9)18)21-14...,0.436372,0.000000,0.394652,0.000000,0.423370,0.000000,0.584091,0.020906,0.532951,0.044444,0.584091,0.060914,HeldOut
50193,1117305,InChI=1S/C18H19Cl2NO4/c1-5-25-18(23)14-10(3)21...,0.481728,0.003484,0.331436,0.000000,0.471541,0.005076,0.725631,0.149826,0.685987,0.244444,0.723294,0.167513,HeldOut
50194,1117305,InChI=1S/C16H11BrN2O/c17-9-5-6-14-11(7-9)12-8-...,0.380949,0.000000,0.279776,0.000000,0.380949,0.000000,0.812100,0.118467,0.697407,0.100000,0.812100,0.142132,HeldOut


In [19]:
list_of_lists_df.to_csv("Similarity_def_mean_train_test_no_Class.csv", index=False)